# Laboratorio 2: Armado de un esquema de aprendizaje automático

En el laboratorio final se espera que puedan poner en práctica los conocimientos adquiridos en el curso, trabajando con un conjunto de datos de clasificación.

El objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

El conjunto de datos a utilizar está en `./data/loan_data.csv`. Si abren el archivo verán que al principio (las líneas que empiezan con `#`) describen el conjunto de datos y sus atributos (incluyendo el atributo de etiqueta o clase).

Se espera que hagan uso de las herramientas vistas en el curso. Se espera que hagan uso especialmente de las herramientas brindadas por `scikit-learn`.

In [12]:
import numpy as np
import pandas as pd

# TODO: Agregar las librerías que hagan falta
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import SGDClassifier
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import f1_score
from sklearn.metrics         import precision_score
from sklearn.metrics         import recall_score
from sklearn.metrics         import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree            import DecisionTreeClassifier

## Carga de datos y división en entrenamiento y evaluación

La celda siguiente se encarga de la carga de datos (haciendo uso de pandas). Estos serán los que se trabajarán en el resto del laboratorio.

In [ ]:
dataset = pd.read_csv("./data/loan_data.csv", comment="#")

# División entre instancias y etiquetas
X, y = dataset.iloc[:, 1:], dataset.TARGET

# división entre entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print (dataset.columns)
print (dataset.shape)
dict = {'a' : 1, 'b' : 2}
description_dictionary = {  'TARGET' : '1: Defaulted loan, 0: Loan Paid'       ,
                            'LOAN'   : 'Amount of the loan requested'          ,
                            'MORTDUE': 'Amount due on existing mortage'        ,
                            'VALUE'  : 'Value of the current propierty'        ,
                            'YOJ'    : 'Years at present job'                  ,
                            'DEROG'  : 'Number of major derogatory reports'    ,
                            "DELINQ" : 'Number of deliquent credit lines'      ,
                            'CLAGE'  : 'Age of the oldest trade line in months',
                            'NINQ'   : 'Number of recent credit lines'         ,
                            "CLNO"   : 'Number of credit lines'                ,
                            'DEBTINC': 'Debt-to-income ratio'
                          }


Documentación:

- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

## Ejercicio 1: Descripción de los Datos y la Tarea

Responder las siguientes preguntas:

1. ¿De qué se trata el conjunto de datos?
2. ¿Cuál es la variable objetivo que hay que predecir? ¿Qué significado tiene?
3. ¿Qué información (atributos) hay disponible para hacer la predicción?
4. ¿Qué atributos imagina ud. que son los más determinantes para la predicción?

**No hace falta escribir código para responder estas preguntas.**

1. Es sobre la automizacion del sistema de otogarcion de creditos por parte del banco, 
   siguendo la Recommendation de Igual de Oportunidad de Acceso a credito.
   El modelo de datos se va a entranar con un conjutn ode datos de aplicaciones a creditos recientes.

2. La variable objetivo es saber si con lo datos obtenidos la persona sera capaz o no de pagar el credito o se transforma en moroso

3. - Loan, es el monto del credito solicitado.
   - Mortue, es lo que se debe de la hipoteca aactual
   - Value, es el valor actual de la propiedad
   - Yoj, la antiguedad en el trabajo actual
   - Derog, la cantidad de incedentes relacionados al pago tarde o algun otro porblema con el pago.
   - Deling, es cuando no se llega a pagar ni el monto minimo exigido a los 30 dias del vencido el prestamo.
   - Clage, es un trackeo de todas las actividades relacionadas con el credito
   - Ning, la cantidad de credit lines recientes
   - Clno, cantidad total de credit lines
   - Debtinc, la relacion en deudas e ingresos


4. Diria que para tomar una desicion, las de mas peso serian:
   - Loan, el monto solicitado
   - Value, valor de la propiedad, si es menor al loan y algo raro hay
   - Debtinc, te dice mas o menos como puede responder al creedito.
   - Yoj, estabilidad laboral
   - Y con las demas casi que se podria armar una especie de reputacion creediticia, unificada en solo uan variable.

## Ejercicio 2: Predicción con Modelos Lineales

En este ejercicio se entrenarán modelos lineales de clasificación para predecir la variable objetivo.

Para ello, deberán utilizar la clase SGDClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/sgd.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html


### Ejercicio 2.1: SGDClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador SGDClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
def print_model_metrics(y_teste, y_pred):
    accuracy    = accuracy_score   (y_test, y_pred) 
    precision   = precision_score  (y_test, y_pred)
    recall      = recall_score     (y_test, y_pred)
    f1          = f1_score         (y_test, y_pred)
    confusion   = confusion_matrix (y_test, y_pred)

    
    metrics_map = { 'Accuracy'         : accuracy   ,
                    'Precision'        : precision  ,
                    'Recall'           : recall     , 
                    'f1_score'         : f1         
                  }
    
    for metric in metrics_map:
        print(metric, metrics_map[metric])
    
    print(confusion)
    
    print('\n True Negatives '  + str(confusion[0][0]))
    print('\n False Negatives ' + str(confusion[1][0]))
    print('\n True Positives '  + str(confusion[1][1]))
    print('\n False Positives ' + str(confusion[0][1]))

In [ ]:
classifier = SGDClassifier(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# Look for model evaluation for more info on the metrics
print_model_metrics(y_test, y_pred)

### Ejercicio 2.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del SGDClassifier. Como mínimo, probar diferentes funciones de loss, tasas de entrenamiento y tasas de regularización.

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
param_map = { 'loss'          : ['hinge','log','squared_hinge','perceptron'],
              'penalty'       : ['l2','l1','elasticnet'],
              'learning_rate' : ['optimal','adaptive','invscaling', 'adaptive'],
              'eta0'          : [0.1, 0.5, 1],
              'max_iter'      : [1000, 1500]
}

In [ ]:
classifier  = SGDClassifier(random_state = 0)
grid_search = GridSearchCV(classifier, param_map)
kf          = KFold(n_splits=5, shuffle=False)

# Una aclaracion importante, al principio del lab se dividieron los datos en 
# train y test, pandas tiene el index, y como los divide random, te quedan mezclados los datos
# El kfold se ve que toma el valor de index y lo intenta dividit con eso
# Te puede dar indices que no existan en el conjunto de datos parcial de train
# pero si en el grupo entero de datos, por eso para este punto nos olvidamos de la division de
# datos en train y test, y pasan todos a train y validacion

for train_index, val_index in kf.split(X):
    X_train_kf, X_val_kf = X.reindex(train_index), X.reindex(val_index)
    y_train_kf, y_val_kf = y.reindex(train_index), y.reindex(val_index)
    if len(y_train_kf.unique()) > 1: #There has to be more than one class, otherwise it throws an error 
        grid_search.fit(X_train_kf, y_train_kf)


In [ ]:
results = grid_search.cv_results_
params  = results['params']
mean    = results['mean_test_score']
std     = results['std_test_score']
print("loss\tpenalty\tlearning_rate\teta0\t| mean\tstd\trank")
for p, m, s in zip(params, mean, std):
    print(f"{p['loss']}\t{p['penalty']}\t{p['learning_rate']}\t{p['eta0']}\t| {m:0.2f}\t{s:0.2f}\t")

In [ ]:
best_model = grid_search.best_estimator_
best_model

In [ ]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print_model_metrics(y_test, y_pred)

## Ejercicio 3: Árboles de Decisión

En este ejercicio se entrenarán árboles de decisión para predecir la variable objetivo.

Para ello, deberán utilizar la clase DecisionTreeClassifier de scikit-learn.

Documentación:
- https://scikit-learn.org/stable/modules/tree.html
  - https://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html

### Ejercicio 3.1: DecisionTreeClassifier con hiperparámetros por defecto

Entrenar y evaluar el clasificador DecisionTreeClassifier usando los valores por omisión de scikit-learn para todos los parámetros. Únicamente **fijar la semilla aleatoria** para hacer repetible el experimento.

Evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


In [15]:
tree = DecisionTreeClassifier(random_state = 0)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

print_model_metrics(y_test, y_pred)

Accuracy 0.8814016172506739
Precision 0.6166666666666667
Recall 0.6379310344827587
f1_score 0.6271186440677966
[[290  23]
 [ 21  37]]

 True Negatives 290

 False Negatives 21

 True Positives 37

 False Positives 23


### Ejercicio 3.2: Ajuste de Hiperparámetros

Seleccionar valores para los hiperparámetros principales del DecisionTreeClassifier. Como mínimo, probar diferentes criterios de partición (criterion), profundidad máxima del árbol (max_depth), y cantidad mínima de samples por hoja (min_samples_leaf).

Para ello, usar grid-search y 5-fold cross-validation sobre el conjunto de entrenamiento para explorar muchas combinaciones posibles de valores.

Reportar accuracy promedio y varianza para todas las configuraciones.

Para la mejor configuración encontrada, evaluar sobre el conjunto de **entrenamiento** y sobre el conjunto de **evaluación**, reportando:
- Accuracy
- Precision
- Recall
- F1
- matriz de confusión


Documentación:
- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [48]:
param_map = { 'criterion'        : ['gini','entropy'],
              'splitter'         : ['best','random'],
              'max_depth'        : range(2,31,5),
              'min_samples_leaf' : range(1,20, 2)
}

In [49]:
tree        = DecisionTreeClassifier(random_state = 0)
grid_search = GridSearchCV(tree, param_map)
kf          = KFold(n_splits=5, shuffle=False)

# Una aclaracion importante, al principio del lab se dividieron los datos en 
# train y test, pandas tiene el index, y como los divide random, te quedan mezclados los datos
# El kfold se ve que toma el valor de index y lo intenta dividit con eso
# Te puede dar indices que no existan en el conjunto de datos parcial de train
# pero si en el grupo entero de datos, por eso para este punto nos olvidamos de la division de
# datos en train y test, y pasan todos a train y validacion

for train_index, val_index in kf.split(X):
    X_train_kf, X_val_kf = X.reindex(train_index), X.reindex(val_index)
    y_train_kf, y_val_kf = y.reindex(train_index), y.reindex(val_index)
    if len(y_train_kf.unique()) > 1: #There has to be more than one class, otherwise it throws an error 
        grid_search.fit(X_train_kf, y_train_kf)


In [45]:
results = grid_search.cv_results_
params  = results['params']
mean    = results['mean_test_score']
std     = results['std_test_score']
print("criterion\splitter\max_depth\min_sample_leaf\t| mean\tstd\trank")
for p, m, s in zip(params, mean, std):
    print(f"{p['criterion']}\t{p['splitter']}\t{p['max_depth']}\t{p['min_samples_leaf']}\t| {m:0.2f}\t{s:0.2f}\t")

criterion\splitter\max_depth\min_sample_leaf	| mean	std	rank
gini	best	2	1	| 0.84	0.02	
gini	random	2	1	| 0.81	0.01	
gini	best	2	2	| 0.84	0.02	
gini	random	2	2	| 0.81	0.01	
gini	best	2	3	| 0.84	0.02	
gini	random	2	3	| 0.81	0.01	
gini	best	2	4	| 0.84	0.02	
gini	random	2	4	| 0.81	0.01	
gini	best	2	5	| 0.84	0.02	
gini	random	2	5	| 0.81	0.01	
gini	best	2	6	| 0.84	0.02	
gini	random	2	6	| 0.81	0.01	
gini	best	2	7	| 0.84	0.02	
gini	random	2	7	| 0.81	0.01	
gini	best	4	1	| 0.83	0.02	
gini	random	4	1	| 0.82	0.02	
gini	best	4	2	| 0.83	0.02	
gini	random	4	2	| 0.82	0.02	
gini	best	4	3	| 0.83	0.02	
gini	random	4	3	| 0.82	0.02	
gini	best	4	4	| 0.83	0.02	
gini	random	4	4	| 0.82	0.02	
gini	best	4	5	| 0.84	0.01	
gini	random	4	5	| 0.82	0.02	
gini	best	4	6	| 0.84	0.01	
gini	random	4	6	| 0.82	0.02	
gini	best	4	7	| 0.84	0.01	
gini	random	4	7	| 0.82	0.02	
gini	best	6	1	| 0.83	0.02	
gini	random	6	1	| 0.82	0.02	
gini	best	6	2	| 0.83	0.01	
gini	random	6	2	| 0.82	0.02	
gini	best	6	3	| 0.83	0.02	
gini	random	6	3	

In [50]:
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=7, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [51]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print_model_metrics(y_test, y_pred)

Accuracy 0.8921832884097035
Precision 0.8
Recall 0.41379310344827586
f1_score 0.5454545454545454
[[307   6]
 [ 34  24]]

 True Negatives 307

 False Negatives 34

 True Positives 24

 False Positives 6
